In [1]:
import os
import numpy as np
from PIL import Image
from functools import reduce
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout,Input,Flatten,Lambda,Dense,BatchNormalization
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Nadam

In [2]:
dir_folder = '/content/drive/MyDrive/datasets/Brain MRI'
print(os.path.exists(dir_folder))
# yes_dir = os.path.join(dir_folder,'yes')
# no_dir = os.path.join(dir_folder,'no')
# def extract_files(base1,base2):
#   yes_files = []
#   no_files = []
#   for i,j in zip(os.listdir(base1),os.listdir(base2)):
#       yes_full_path = os.path.join(base1,i)
#       no_full_path = os.path.join(base2,j)
#       yes_files.append(yes_full_path)
#       no_files.append(no_full_path)
#   return yes_files,no_files
# yes_files,no_files = extract_files(yes_dir,no_dir)

True


In [3]:
# def load_image(files,label_value):
#   images = []
#   labels = [label_value] * len(files)
#   for file in yes_files:
#       img = Image.open(file)
#       img = img.convert('L')
#       img = img.resize((224, 224))
#       img_array = np.array(img)
#       images.append(img_array)

#   result = np.stack(images, axis=0)

#   return result,labels

# # no_images,labels = load_image(no_files,0)
# yes_images,labels = load_image(yes_files,1)

In [4]:

train_gen = ImageDataGenerator(rescale=1./255,
                               validation_split=0.2,
                               rotation_range=40,
                               height_shift_range=0.2,
                               width_shift_range=0.2,
                               shear_range=0.2,
                               zoom_range=0.2,
                               fill_mode='nearest',
                               horizontal_flip=True)

In [5]:
train_generator = train_gen.flow_from_directory(dir_folder,
                                                target_size=(224,224),
                                                batch_size=32,
                                                class_mode='categorical',
                                                subset='training')

Found 47 images belonging to 2 classes.


In [6]:
gray_scale_layer = Lambda(lambda x: tf.image.rgb_to_grayscale(x))

model = Sequential([
    gray_scale_layer,
    Conv2D(32,(3,3),activation='relu',input_shape=(224,224,1)),
    MaxPooling2D((2,2)),
    BatchNormalization(),
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    BatchNormalization(),
    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D((2,2)),
    BatchNormalization(),
    Conv2D(256,(3,3),activation='relu',),
    MaxPooling2D((2,2)),
    BatchNormalization(),
    Flatten(),
    Dense(512,activation='relu'),
    BatchNormalization(),
    Dense(256,activation='relu'),
    BatchNormalization(),
    Dense(2,activation='softmax')
])
input_shape = Input(shape=(224,224,3))
Model(inputs=input_shape,outputs=model(input_shape))
model.compile(loss='categorical_crossentropy',optimizer=Nadam(learning_rate=0.01),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)

epochs=20
model.fit(train_generator,
          callbacks=[early_stopping],
          steps_per_epoch=len(train_generator) // 32,
          epochs=epochs,
          )

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 21s 7s/step - accuracy: 0.6619 - loss: 1.3760
Epoch 2/20


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.5988 - loss: 2.7863 
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.4512 - loss: 2.7734 
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.6339 - loss: 0.9645 
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.7957 - loss: 0.5598 
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.8057 - loss: 0.5331
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.8199 - loss: 0.3776
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.8382 - loss: 0.4199
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.8421 - loss: 0.4162
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.8785 - loss: 0.3074
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.8941 - loss: 0.4036 
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.9210 - loss: 0.2625
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.9433 - loss: 0.2759
Epoch 

In [7]:
classes = {
    0:'no',
    1:'yes'
}

In [8]:
def predict(file):
  img = Image.open(file)
  img = img.resize((224, 224))
  # img = img.convert('L')
  img = img.convert('RGB')  # Convert to RGB to match model input
  img = np.array(img)
  # Add a batch dimension
  img = np.expand_dims(img, axis=0)
  # Preprocess using the same function used for training
  # Rescale the image
  img = img / 255.0
  prediction = model.predict(img)
  return classes[np.argmax(prediction)]

file = '/content/drive/MyDrive/datasets/Brain MRI/yes/Y10.jpg'
predict(file)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


'yes'

In [10]:
model.save('/content/drive/MyDrive/models/model.keras')